<a href="https://colab.research.google.com/github/sulochandhungel/PythonTools/blob/main/NOAA_AWOS_Metar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import division
import os
import datetime

import numpy as np
import pandas
#import statsmodels.api as sm

import matplotlib
matplotlib.rcParams['savefig.dpi'] = 144
import matplotlib.pyplot as plt

#!pip install metar
import metar

from IPython.display import HTML
%config InlineBackend.figure_format = 'svg'

## uncomment the next line to embed the FAA map into the notebook.
#HTML('')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 KB 6.5 MB/s eta 0:00:00


In [3]:
from metar import Metar
obs = Metar.Metar('METAR KEWR 111851Z VRB03G19KT 2SM R04R/3000VP6000FT TSRA BR FEW015 BKN040CB BKN065 OVC200 22/22 A2987 RMK AO2 PK WND 29028/1817 WSHFT 1812 TSB05RAB22 SLP114 FRQ LTGICCCCG TS OHD AND NW -N-E MOV NE P0013 T02270215')
print (obs.string())

station: KEWR
type: routine report, cycle 19 (automatic report)
time: Wed Jan 11 18:51:00 2023
temperature: 22.7 C
dew point: 21.5 C
wind: variable at 3 knots, gusting to 19 knots
peak wind: WNW at 28 knots at 18:17
wind shift: 18:12
visibility: 2 miles
visual range: on runway 04R, from 3000 to greater than 6000 feet
pressure: 1011.5 mb
weather: thunderstorm with rain; mist
sky: a few clouds at 1500 feet
     broken cumulonimbus at 4000 feet
     broken clouds at 6500 feet
     overcast at 20000 feet
sea-level pressure: 1011.4 mb
1-hour precipitation: 0.13in
remarks:
- Automated station (type 2)
- peak wind 28kt from 290 degrees at 18:17
- wind shift at 18:12
- frequent lightning (intracloud,cloud-to-cloud,cloud-to-ground)
- thunderstorm overhead and NW
- TSB05RAB22 -N-E MOV NE
METAR: METAR KEWR 111851Z VRB03G19KT 2SM R04R/3000VP6000FT TSRA BR FEW015 BKN040CB BKN065 OVC200 22/22 A2987 RMK AO2 PK WND 29028/1817 WSHFT 1812 TSB05RAB22 SLP114 FRQ LTGICCCCG TS OHD AND NW -N-E MOV NE P0013 T

In [36]:
dir(obs)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_day',
 '_do_trend_handlers',
 '_handleAutoRemark',
 '_handleColor',
 '_handleCorrection',
 '_handleIceAccretionRemark',
 '_handleLightningRemark',
 '_handleModifier',
 '_handlePeakWindRemark',
 '_handlePrecip1hrRemark',
 '_handlePrecip24hrRemark',
 '_handlePress3hrRemark',
 '_handlePressure',
 '_handleRecent',
 '_handleRunway',
 '_handleRunwayState',
 '_handleSealvlPressRemark',
 '_handleSky',
 '_handleSnowDepthRemark',
 '_handleStation',
 '_handleTSLocRemark',
 '_handleTemp',
 '_handleTemp1hrRemark',
 '_handleTemp24hrRemark',
 '_handleTemp6hrRemark',
 '_handleTime',
 '_handleTrend',
 '_handleType',
 '_handleVisibility'

In [67]:
# COPY THE METAR DATA from the website here:

raw_data = """\
KSHN 060053Z AUTO 24016G23KT 10SM CLR 08/06 A2998 RMK AO2 PK WND 24027/0019 SLP155 T00780056
KSHN 052353Z AUTO 24014KT 10SM FEW016 BKN080 10/07 A2994 RMK AO2 SLP141 60002 T01000072 10106 20072 53022
KSHN 052253Z AUTO 24016G20KT 10SM FEW015 SCT065 OVC080 09/07 A2990 RMK AO2 SLP127 T00940072
KSHN 052241Z AUTO 23014KT 10SM FEW012 BKN070 OVC080 10/07 A2990 RMK AO2 T01000072
KSHN 052200Z AUTO 21009KT 10SM FEW009 SCT012 BKN020 10/08 A2988 RMK AO2 T01000078
KSHN 052153Z AUTO 21009KT 10SM SCT011 BKN065 BKN100 10/08 A2988 RMK AO2 RAE04 SLP119 P0000 T01000083
KSHN 052150Z AUTO 20009KT 10SM FEW007 BKN065 BKN100 10/08 A2987 RMK AO2 RAE04 P0000
KSHN 052122Z AUTO 19006KT 10SM FEW027 OVC070 09/08 A2987 RMK AO2 RAE04 P0000 T00940078
KSHN 052102Z AUTO 22005KT 10SM -RA SCT011 OVC029 09/07 A2987 RMK AO2 P0000 T00940072
KSHN 052053Z AUTO 24004KT 10SM -RA BKN011 OVC030 09/08 A2987 RMK AO2 RAB53 SLP116 P0000 60002 T00940078 53015
KSHN 052035Z AUTO 13003KT 10SM BKN011 BKN021 OVC027 09/08 A2986 RMK AO2 CIG 007V014 T00890078
KSHN 052015Z AUTO 00000KT 10SM SCT009 BKN016 OVC025 09/07 A2985 RMK AO2 CIG 013V022 T00890072
KSHN 051953Z AUTO 00000KT 10SM BKN009 OVC070 09/07 A2984 RMK AO2 RAE02 SLP109 P0000 T00890072
KSHN 051902Z AUTO 00000KT 10SM FEW006 BKN031 OVC065 08/07 A2983 RMK AO2 RAE02 P0000 T00830072
KSHN 051853Z AUTO 00000KT 10SM -RA FEW023 BKN031 OVC060 08/07 A2982 RMK AO2 SLP102 P0002 T00830072
KSHN 051848Z AUTO 00000KT 10SM -RA SCT025 BKN031 OVC055 08/07 A2982 RMK AO2 P0002
KSHN 051804Z AUTO 00000KT 10SM -RA BKN025 OVC035 07/07 A2980 RMK AO2 P0001 T00720067
KSHN 051753Z AUTO 00000KT 9SM -RA FEW024 SCT030 OVC035 07/07 A2979 RMK AO2 SLP093 P0004 60023 T00720067 10072 20044 51039
KSHN 051653Z AUTO 00000KT 5SM -RA BR OVC037 06/06 A2976 RMK AO2 SLP081 P0008 T00610061
KSHN 051553Z AUTO 15003KT 7SM -RA FEW030 OVC039 06/05 A2972 RMK AO2 SLP068 P0005 T00560050
KSHN 051453Z AUTO 33003KT 9SM -RA OVC043 07/06 A2968 RMK AO2 SLP053 P0003 60006 T00670056 53016
KSHN 051353Z AUTO 00000KT 10SM -RA OVC045 06/05 A2966 RMK AO2 SLP048 P0002 T00560050
KSHN 051253Z AUTO 00000KT 10SM -RA OVC049 07/05 A2964 RMK AO2 RAB1159 SLP039 P0001 T00670050
KSHN 051153Z AUTO 00000KT 10SM OVC055 05/04 A2963 RMK AO2 SLP038 70027 T00500044 10072 20044 55003
KSHN 051053Z AUTO 04003KT 10SM BKN085 06/05 A2963 RMK AO2 SLP035 T00610050
KSHN 050953Z AUTO 03006KT 10SM OVC095 07/05 A2963 RMK AO2 SLP037 T00670050
KSHN 050853Z AUTO 02006KT 10SM SCT100 06/05 A2964 RMK AO2 SLP041 T00610050 56031
KSHN 050753Z AUTO 04004KT 10SM SCT095 06/06 A2967 RMK AO2 SLP052 T00610056 400940061
KSHN 050653Z AUTO 36008KT 10SM BKN085 OVC110 07/06 A2969 RMK AO2 SLP057 T00720061
KSHN 050553Z AUTO 02007KT 10SM BKN100 07/06 A2974 RMK AO2 RAE00 SLP073 P0000 60013 T00670061 10089 20067 58033
KSHN 050453Z AUTO 03009KT 10SM -RA FEW048 BKN055 OVC070 07/06 A2977 RMK AO2 PRESFR SLP083 P0002 T00720061
KSHN 050353Z AUTO 02004KT 10SM -RA OVC055 08/07 A2981 RMK AO2 SLP099 P0002 T00780067
KSHN 050253Z AUTO 06003KT 6SM -RA BR OVC039 08/07 A2983 RMK AO2 SLP106 P0006 60009 T00780072 57012
KSHN 050219Z AUTO 00000KT 6SM -RA BR FEW026 SCT031 OVC037 08/07 A2985 RMK AO2 P0002 T00780072
KSHN 050153Z AUTO 00000KT 10SM -RA BKN027 BKN034 OVC043 08/07 A2985 RMK AO2 SLP111 P0001 T00780072
KSHN 050053Z AUTO 01003KT 9SM -RA OVC049 08/08 A2985 RMK AO2 RAB2357 SLP113 P0002 T00830078
KSHN 042353Z AUTO 36003KT 10SM BKN065 OVC090 09/08 A2987 RMK AO2 SLP117 60006 T00890078 10094 20072 56023
KSHN 042253Z AUTO 01003KT 10SM FEW005 OVC090 09/08 A2988 RMK AO2 SLP122 T00890078
KSHN 042203Z AUTO 35003KT 10SM FEW004 OVC095 09/08 A2991 RMK AO2 T00890078
KSHN 042153Z AUTO 02003KT 10SM OVC100 09/08 A2991 RMK AO2 SLP130 T00890078
KSHN 042053Z AUTO 01005KT 10SM FEW048 BKN090 OVC110 08/08 A2994 RMK AO2 RAE40 SLP140 P0002 60006 T00830078 58016
KSHN 041953Z AUTO 36004KT 10SM -RA OVC050 08/08 A2998 RMK AO2 RAB00 SLP153 P0003 T00830078
KSHN 041853Z AUTO 01003KT 10SM OVC050 08/07 A2999 RMK AO2 RAB16E52 SLP157 P0001 T00780072
KSHN 041753Z AUTO 00000KT 10SM OVC050 07/07 A2998 RMK AO2 SLP156 60008 T00720067 10072 20061 56002
KSHN 041653Z AUTO 00000KT 10SM FEW004 BKN036 OVC046 07/07 A2998 RMK AO2 RAB39E50 SLP156 P0000 T00720067
KSHN 041556Z AUTO 35003KT 10SM SCT006 OVC035 07/06 A2998 RMK AO2 T00670061
KSHN 041553Z AUTO 00000KT 10SM BKN005 OVC035 07/06 A2998 RMK AO2 SLP157 T00670061
KSHN 041542Z AUTO 00000KT 10SM BKN005 OVC035 07/06 A2998 RMK AO2 T00670061
KSHN 041519Z AUTO 00000KT 10SM OVC004 07/06 A2999 RMK AO2 T00670061
KSHN 041506Z AUTO 36003KT 10SM OVC005 07/06 A2999 RMK AO2 T00670061
KSHN 041453Z AUTO 01004KT 10SM OVC006 07/06 A2999 RMK AO2 RAE06 SLP159 P0000 60008 T00670061 56009
KSHN 041414Z AUTO 02004KT 10SM BKN008 OVC034 07/06 A2999 RMK AO2 RAE06 P0000 T00670061
KSHN 041406Z AUTO 03004KT 10SM BKN010 OVC032 07/06 A2999 RMK AO2 RAE06 P0000 T00670061
KSHN 041353Z AUTO 03003KT 4SM -RA BR SCT007 SCT011 OVC030 07/06 A2999 RMK AO2 RAE39B47 SLP161 P0005 T00670061
KSHN 041339Z AUTO 35004KT 7SM FEW006 SCT012 OVC035 06/06 A3000 RMK AO2 RAE39 P0005 T00610061
KSHN 041253Z AUTO 00000KT 3SM -RA BR OVC037 06/06 A3000 RMK AO2 SLP164 P0003 T00610061
KSHN 041153Z AUTO 00000KT 6SM -RA BR SCT026 OVC041 07/07 A3002 RMK AO2 SLP167 P0003 60013 70062 T00720067 10078 20067 58003
KSHN 041121Z AUTO 00000KT 8SM -RA BKN024 OVC060 07/06 A3002 RMK AO2 P0001 T00670061
KSHN 041053Z AUTO 11003KT 9SM -RA FEW018 BKN055 OVC070 07/06 A3002 RMK AO2 SLP170 P0002 T00670061
KSHN 040953Z AUTO 00000KT 10SM -RA FEW029 OVC039 07/07 A3003 RMK AO2 RAB0859 SLP173 P0002 T00670067
KSHN 040853Z AUTO 17004KT 10SM FEW020 SCT024 OVC044 07/06 A3003 RMK AO2 RAE42 SLP172 P0001 60006 T00720061 53007
KSHN 040753Z AUTO 15006KT 7SM -RA FEW019 SCT028 OVC039 07/07 A3002 RMK AO2 RAB09 SLP169 P0003 T00720067 400940000
KSHN 040729Z AUTO 13005KT 6SM -RA BR FEW020 FEW028 OVC041 07/06 A3002 RMK AO2 RAB09 P0002 T00670061
KSHN 040704Z AUTO 17005KT 10SM SCT020 BKN026 OVC080 08/07 A3001 RMK AO2 T00780067
KSHN 040653Z AUTO 17005KT 10SM SCT024 BKN040 OVC080 08/07 A3001 RMK AO2 RAE19 SLP164 P0002 T00780067
KSHN 040640Z AUTO 18006KT 10SM SCT020 SCT060 OVC085 08/07 A3001 RMK AO2 RAE19 P0002 T00780067
KSHN 040553Z AUTO 19005KT 8SM -RA BKN016 BKN026 OVC033 08/07 A3000 RMK AO2 RAB09 SLP164 P0002 60023 T00780067 10083 20078 51020
KSHN 040527Z AUTO 17008KT 8SM -RA FEW017 BKN024 OVC055 08/07 A3000 RMK AO2 RAB09 P0000 T00780067
KSHN 040500Z AUTO 21009KT 10SM FEW017 SCT023 BKN060 08/06 A2999 RMK AO2 T00830061
KSHN 040453Z AUTO 21009KT 10SM SCT016 BKN023 BKN031 08/06 A2999 RMK AO2 SLP159 T00830061
KSHN 040353Z AUTO 23011KT 10SM BKN017 BKN024 OVC045 08/06 A2997 RMK AO2 PK WND 22026/0314 SLP153 T00780061
KSHN 040341Z AUTO 23010G21KT 10SM SCT017 BKN029 OVC046 08/06 A2997 RMK AO2 PK WND 22026/0314 T00830061
KSHN 040253Z AUTO 21013G31KT 10SM SCT019 SCT041 08/06 A2995 RMK AO2 PK WND 22031/0246 RAE19 SLP144 P0000 60021 T00780061 51015
KSHN 040248Z AUTO 22015G31KT 10SM SCT019 SCT026 BKN041 08/06 A2994 RMK AO2 PK WND 22031/0246 RAE19 P0000
KSHN 040224Z AUTO 22015G25KT 9SM SCT018 BKN025 OVC050 08/07 A2994 RMK AO2 PK WND 24029/0204 RAE19 P0000 T00780067
KSHN 040153Z AUTO 22014G29KT 8SM -RA FEW015 SCT019 OVC041 08/06 A2993 RMK AO2 PK WND 21031/0104 SLP139 P0008 T00780061
KSHN 040053Z AUTO 20014G20KT 4SM -RA BR FEW018 BKN022 OVC032 08/07 A2990 RMK AO2 PK WND 20039/0016 SLP129 P0013 T00780067
KSHN 032353Z AUTO 20017G28KT 3SM -RA BR FEW012 SCT024 OVC031 08/06 A2988 RMK AO2 PK WND 18033/2311 SLP123 P0010 60026 T00780061 10094 20044 51016
KSHN 032348Z AUTO 21016G28KT 3SM -RA BR FEW012 BKN026 OVC031 08/06 A2987 RMK AO2 PK WND 18033/2311 P0010
KSHN 032253Z AUTO 18016G27KT 4SM RA BR FEW022 BKN037 OVC047 07/06 A2984 RMK AO2 PK WND 18027/2237 SLP110 P0012 T00720056
KSHN 032242Z AUTO 18015G27KT 5SM -RA BR FEW018 BKN037 OVC047 07/05 A2984 RMK AO2 PK WND 18027/2237 P0011 T00720050
KSHN 032153Z AUTO 17010G19KT 6SM -RA BR SCT023 BKN029 OVC040 07/05 A2983 RMK AO2 SLP106 P0003 T00720050
KSHN 032144Z AUTO 17010KT 6SM -RA BR FEW022 BKN030 OVC039 07/05 A2983 RMK AO2 P0003 T00720050
KSHN 032120Z AUTO 16011KT 8SM -RA BKN028 OVC033 07/05 A2983 RMK AO2 P0001 T00720050
KSHN 032053Z AUTO 16012KT 7SM -RA FEW024 OVC032 07/04 A2983 RMK AO2 PK WND 17030/2001 RAB20 SLP105 P0001 60001 T00720044 56002
KSHN 032044Z AUTO 17010KT 7SM -RA BKN027 OVC032 07/04 A2983 RMK AO2 PK WND 17030/2001 RAB20 P0001 T00720044
KSHN 032037Z AUTO 18013G26KT 8SM -RA FEW023 OVC031 08/04 A2984 RMK AO2 PK WND 17030/2001 RAB20 P0000 T00780039
KSHN 032028Z AUTO 18016G28KT 10SM -RA BKN029 OVC036 08/03 A2983 RMK AO2 PK WND 17030/2001 RAB20 P0000 T00780033
KSHN 031953Z AUTO 16018G29KT 10SM BKN043 OVC070 09/02 A2983 RMK AO2 PK WND 16042/1908 SLP105 T00890017
KSHN 031853Z AUTO 14019G32KT 10SM OVC050 09/01 A2982 RMK AO2 PK WND 17032/1852 RAB1756E12 SLP101 P0000 T00940006
KSHN 031753Z AUTO 12003KT 10SM BKN060 OVC080 04/01 A2984 RMK AO2 SLP108 T00440006 10044 20006 53015
KSHN 031653Z AUTO 00000KT 10SM BKN100 03/00 A2981 RMK AO2 SLP098 T00330000
KSHN 031553Z AUTO 00000KT 10SM SCT120 02/M01 A2981 RMK AO2 SLP098 T00221006
KSHN 031453Z AUTO 00000KT 10SM OVC120 02/M01 A2979 RMK AO2 SLP093 T00221006 56022
KSHN 031353Z AUTO 00000KT 10SM OVC100 03/M01 A2979 RMK AO2 SLP094 T00281006
KSHN 031253Z AUTO 34004KT 10SM BKN110 02/M01 A2984 RMK AO2 PRESFR SLP108 T00221006
KSHN 031153Z AUTO 00000KT 10SM CLR 01/M01 A2986 RMK AO2 SLP114 T00061011 10056 20000 58018
KSHN 031053Z AUTO 23003KT 10SM CLR 01/M01 A2987 RMK AO2 SLP120 T00111006
KSHN 030953Z AUTO 08005KT 10SM BKN110 04/00 A2989 RMK AO2 SLP126 T00390000
KSHN 030853Z AUTO 34008KT 10SM BKN110 06/M01 A2991 RMK AO2 SLP132 T00561006 58033
KSHN 030753Z AUTO 35003KT 10SM CLR 01/M01 A2996 RMK AO2 SLP150 T00061011 401001056
KSHN 030653Z AUTO 33003KT 10SM CLR 01/M01 A2998 RMK AO2 SLP156 T00111006
KSHN 030553Z AUTO 00000KT 10SM FEW120 01/M01 A3001 RMK AO2 SLP166 T00111011 10089 20006 58016
KSHN 030453Z AUTO 00000KT 10SM CLR 01/M01 A3004 RMK AO2 SLP177 T00111011
KSHN 030353Z AUTO 00000KT 10SM BKN100 OVC110 02/M01 A3004 RMK AO2 SLP181 T00171011
KSHN 030253Z AUTO 00000KT 10SM CLR 01/M02 A3005 RMK AO2 SLP183 T00111017 56012
KSHN 030153Z AUTO 00000KT 10SM CLR 03/M01 A3006 RMK AO2 SLP186 T00331006
KSHN 030053Z AUTO 00000KT 10SM CLR 07/M02 A3007 RMK AO2 SLP189 T00721022
KSHN 022353Z AUTO 10004KT 10SM CLR 09/M03 A3009 RMK AO2 SLP193 T00891028 10100 20006 56010
KSHN 022253Z AUTO 11004KT 10SM CLR 09/M02 A3009 RMK AO2 SLP193 T00941022
KSHN 022153Z AUTO 10005KT 10SM CLR 09/M02 A3010 RMK AO2 SLP197 T00941017
KSHN 022053Z AUTO 09005KT 10SM CLR 08/M01 A3012 RMK AO2 SLP203 T00831011 58010
KSHN 021953Z AUTO 11004KT 10SM CLR 08/M02 A3013 RMK AO2 SLP208 T00781017
KSHN 021853Z AUTO 00000KT 10SM CLR 04/01 A3014 RMK AO2 SLP214 T00440006
KSHN 021753Z AUTO 00000KT 10SM CLR 01/M01 A3015 RMK AO2 SLP215 I6001 T00061006 10006 21056 53010
KSHN 021653Z AUTO 00000KT 10SM CLR M02/M03 A3013 RMK AO2 SLP209 I1001 T10171028
KSHN 021553Z AUTO 00000KT 10SM CLR M04/M05 A3012 RMK AO2 SLP208 I1000 T10441050
KSHN 021453Z AUTO 00000KT 10SM CLR M05/M06 A3012 RMK AO2 SLP205 I1000 I3000 T10501056 55001 $
KSHN 021353Z AUTO 00000KT 10SM CLR M03/M04 A3011 RMK AO2 SLP203 I1000 T10331039
KSHN 021325Z AUTO 00000KT 10SM FEW001 M03/M04 A3011 RMK AO2 T10331044
KSHN 021253Z AUTO 00000KT 10SM CLR M04/M04 A3011 RMK AO2 SLP203 I1000 T10391044
KSHN 021153Z AUTO 00000KT 10SM CLR M04/M05 A3012 RMK AO2 SLP204 I1000 I6001 T10391050 11017 21044 50002
KSHN 021053Z AUTO 24003KT 10SM CLR M04/M04 A3012 RMK AO2 SLP205 I1000 T10391044
KSHN 020953Z AUTO 30003KT 3SM BR CLR M04/M05 A3012 RMK AO2 SLP205 I1000 T10391050
KSHN 020951Z AUTO 28003KT 2 1/2SM BR CLR M04/M04 A3012 RMK AO2 VIS 1V5 I1000
KSHN 020941Z AUTO 00000KT 3/4SM BR CLR M04/M04 A3012 RMK AO2 I1000 T10391044
KSHN 020924Z AUTO 00000KT 1 3/4SM BR CLR M03/M04 A3012 RMK AO2 T10331039
KSHN 020909Z AUTO 00000KT 3SM BR CLR M03/M04 A3011 RMK AO2 T10331044
KSHN 020853Z AUTO 00000KT 1SM BR CLR M03/M04 A3011 RMK AO2 VIS 1/2V2 1/2 SLP202 I1001 I3001 T10331039 50000
KSHN 020846Z AUTO 00000KT 1 3/4SM BR CLR M03/M04 A3011 RMK AO2 I1001 T10331044
KSHN 020753Z AUTO 00000KT 8SM CLR M03/M03 A3012 RMK AO2 SLP205 T10281033 400891033
KSHN 020747Z AUTO 00000KT 3SM BR CLR M03/M03 A3012 RMK AO2
KSHN 020735Z AUTO 00000KT 1 3/4SM BR CLR M03/M04 A3013 RMK AO2 T10331039
KSHN 020653Z AUTO 00000KT 10SM CLR M03/M03 A3012 RMK AO2 SLP205 T10281033
KSHN 020553Z AUTO 00000KT 9SM CLR M03/M03 A3011 RMK AO2 SLP204 T10281028 10089 21028 52004
KSHN 020453Z AUTO 00000KT 10SM CLR M03/M03 A3011 RMK AO2 SLP203 T10281033
KSHN 020353Z AUTO 00000KT 10SM CLR M01/M02 A3010 RMK AO2 SLP201 T10111022
KSHN 020253Z AUTO 36003KT 10SM CLR 02/M01 A3010 RMK AO2 SLP199 T00171006 55000
KSHN 020153Z AUTO 00000KT 10SM CLR 02/M01 A3009 RMK AO2 SLP195 T00171006
KSHN 020053Z AUTO 00000KT 10SM CLR 07/00 A3009 RMK AO2 SLP195 T00720000
KSHN 012353Z AUTO 00000KT 10SM CLR 09/00 A3010 RMK AO2 SLP198 T00890000 10089 20017 55003
KSHN 012253Z AUTO 12004KT 10SM CLR 08/01 A3010 RMK AO2 SLP196 T00830006
KSHN 012153Z AUTO 11004KT 10SM CLR 08/01 A3010 RMK AO2 SLP197 T00830006
KSHN 012053Z AUTO 14003KT 10SM CLR 07/01 A3011 RMK AO2 SLP201 T00720006 58005
KSHN 011953Z AUTO 00000KT 10SM CLR 06/00 A3013 RMK AO2 SLP206 T00560000
KSHN 011853Z AUTO 00000KT 10SM CLR 04/M01 A3013 RMK AO2 SLP210 T00391011
KSHN 011753Z AUTO 00000KT 10SM CLR 02/M01 A3013 RMK AO2 SLP208 T00171011 10017 21006 52001
KSHN 011653Z AUTO 00000KT 10SM BKN035 01/M01 A3013 RMK AO2 SLP208 T00111006
KSHN 011553Z AUTO 00000KT 10SM OVC038 M01/M01 A3012 RMK AO2 SLP208 T10061011
KSHN 011453Z AUTO 00000KT 10SM OVC039 M01/M01 A3012 RMK AO2 SLP207 T10061011 55005
KSHN 011353Z AUTO 00000KT 10SM BKN040 00/M01 A3012 RMK AO2 SLP204 T00001011
KSHN 011253Z AUTO 00000KT 10SM OVC038 M01/M01 A3013 RMK AO2 SLP208 T10061011
KSHN 011153Z AUTO 00000KT 10SM OVC039 M01/M01 A3014 RMK AO2 SLP211 T10061011 10006 21011 50000
KSHN 011053Z AUTO 00000KT 10SM SCT029 SCT044 M01/M02 A3014 RMK AO2 SLP211 T10061017
KSHN 011049Z AUTO 00000KT 10SM SCT029 00/M02 A3014 RMK AO2
KSHN 011041Z AUTO 00000KT 10SM BKN029 BKN040 00/M02 A3014 RMK AO2 T00001017
KSHN 011003Z AUTO 00000KT 10SM FEW030 OVC040 00/M02 A3014 RMK AO2 T00001017
KSHN 010953Z AUTO 00000KT 10SM BKN029 BKN035 OVC042 00/M02 A3014 RMK AO2 SLP211 T00001017
KSHN 010853Z AUTO 00000KT 10SM FEW030 OVC040 00/M02 A3013 RMK AO2 SLP210 T00001017 58001
KSHN 010753Z AUTO 36003KT 10SM OVC037 00/M02 A3014 RMK AO2 SLP213 T00001017 400441044
KSHN 010653Z AUTO 00000KT 10SM OVC037 M01/M02 A3015 RMK AO2 SLP214 T10061017
KSHN 010553Z AUTO 00000KT 10SM BKN037 M01/M02 A3014 RMK AO2 SLP211 T10111022 10044 21011 58001
KSHN 010453Z AUTO 00000KT 10SM CLR M01/M02 A3014 RMK AO2 SLP213 T10111022
KSHN 010353Z AUTO 23003KT 10SM SCT110 M01/M02 A3014 RMK AO2 SLP214 T10061022
KSHN 010253Z AUTO 00000KT 10SM FEW110 01/M02 A3014 RMK AO2 SLP212 T00111017 56012
KSHN 010153Z AUTO 00000KT 10SM BKN100 02/M02 A3015 RMK AO2 SLP215 T00171017
"""


In [51]:
raw_data = """\
KSHN 060053Z AUTO 24016G23KT 10SM CLR 08/06 A2998 RMK AO2 PK WND 24027/0019 SLP155 T00780056
KSHN 052353Z AUTO 24014KT 10SM FEW016 BKN080 10/07 A2994 RMK AO2 SLP141 60002 T01000072 10106 20072 53022
KSHN 052253Z AUTO 24016G20KT 10SM FEW015 SCT065 OVC080 09/07 A2990 RMK AO2 SLP127 T00940072
KSHN 052241Z AUTO 23014KT 10SM FEW012 BKN070 OVC080 10/07 A2990 RMK AO2 T01000072
KSHN 052200Z AUTO 21009KT 10SM FEW009 SCT012 BKN020 10/08 A2988 RMK AO2 T01000078
KSHN 052153Z AUTO 21009KT 10SM SCT011 BKN065 BKN100 10/08 A2988 RMK AO2 RAE04 SLP119 P0000 T01000083
KSHN 052150Z AUTO 20009KT 10SM FEW007 BKN065 BKN100 10/08 A2987 RMK AO2 RAE04 P0000
KSHN 052122Z AUTO 19006KT 10SM FEW027 OVC070 09/08 A2987 RMK AO2 RAE04 P0000 T00940078
KSHN 052102Z AUTO 22005KT 10SM -RA SCT011 OVC029 09/07 A2987 RMK AO2 P0000 T00940072
KSHN 052053Z AUTO 24004KT 10SM -RA BKN011 OVC030 09/08 A2987 RMK AO2 RAB53 SLP116 P0000 60002 T00940078 53015
KSHN 052035Z AUTO 13003KT 10SM BKN011 BKN021 OVC027 09/08 A2986 RMK AO2 CIG 007V014 T00890078
KSHN 052015Z AUTO 00000KT 10SM SCT009 BKN016 OVC025 09/07 A2985 RMK AO2 CIG 013V022 T00890072
KSHN 051953Z AUTO 00000KT 10SM BKN009 OVC070 09/07 A2984 RMK AO2 RAE02 SLP109 P0000 T00890072
KSHN 051902Z AUTO 00000KT 10SM FEW006 BKN031 OVC065 08/07 A2983 RMK AO2 RAE02 P0000 T00830072
KSHN 051853Z AUTO 00000KT 10SM -RA FEW023 BKN031 OVC060 08/07 A2982 RMK AO2 SLP102 P0002 T00830072
KSHN 051848Z AUTO 00000KT 10SM -RA SCT025 BKN031 OVC055 08/07 A2982 RMK AO2 P0002
KSHN 051804Z AUTO 00000KT 10SM -RA BKN025 OVC035 07/07 A2980 RMK AO2 P0001 T00720067
KSHN 051753Z AUTO 00000KT 9SM -RA FEW024 SCT030 OVC035 07/07 A2979 RMK AO2 SLP093 P0004 60023 T00720067 10072 20044 51039
KSHN 051653Z AUTO 00000KT 5SM -RA BR OVC037 06/06 A2976 RMK AO2 SLP081 P0008 T00610061
"""

In [162]:
from metar import Metar
import pandas as pd

DT_list = []
precip_1hr = []
precip_24hr = []
precip_3hr = []
precip_6hr = []

raw_data_splitlines = raw_data.splitlines()
for rawtxt in raw_data_splitlines:
  #print (rawtxt)
  par_text = Metar.Metar(rawtxt)
  DT_list.append(par_text.time)
  precip_1hr.append(str(par_text.precip_1hr).split("in")[0])
  precip_24hr.append(str(par_text.precip_24hr).split("in")[0])
  precip_3hr.append(str(par_text.precip_3hr).split("in")[0])
  precip_6hr.append(str(par_text.precip_6hr).split("in")[0])

#precip_1hr_ = [None if i == "None" else float(i) for i in precip_1hr]
precip_1hr_ = [0 if i == "None" else float(i) for i in precip_1hr]
precip_3hr_ = [0 if i == "None" else float(i) for i in precip_3hr]
precip_6hr_ = [0 if i == "None" else float(i) for i in precip_6hr]
precip_24hr_ = [0 if i == "None" else float(i) for i in precip_24hr]

output_df = pd.DataFrame({'DateTime': DT_list,
                          'Precip_1hr_in': precip_1hr_,
                          'Precip_3hr_in': precip_3hr_,
                          'Precip_6hr_in': precip_6hr_,
                          'Precip_24hr_in': precip_24hr_,
                          })

output_df = output_df.sort_values(by='DateTime')

# Resampled
output_df_rs = output_df.set_index('DateTime').resample('H').mean().interpolate('linear')


In [169]:
#output_df_rs

In [170]:
import plotly.express as px
fig = px.line(output_df, x = 'DateTime',y = output_df.columns, markers=False, title = 'Precip')
fig.show()


In [171]:
import plotly.express as px
output_df_rs = output_df.set_index('DateTime').resample('30Min').mean().interpolate('linear')
fig = px.line(output_df_rs, x = output_df_rs.index,y = output_df_rs.columns, markers=True, title = 'Precip Resampled')
fig.show()

In [172]:
import plotly.express as px
output_df_rs = output_df.set_index('DateTime').resample('3H').mean().interpolate('linear')

output_df_rs['Precip_1hr_in'] = output_df_rs['Precip_1hr_in'].cumsum()
output_df_rs['Precip_3hr_in'] = output_df_rs['Precip_3hr_in'].cumsum()
output_df_rs['Precip_6hr_in'] = output_df_rs['Precip_6hr_in'].cumsum()
output_df_rs['Precip_24hr_in'] = output_df_rs['Precip_24hr_in'].cumsum()

fig = px.line(output_df_rs, x = output_df_rs.index,y = output_df_rs.columns, markers=False, title = 'Cumulative Precip Resampled')
fig.show()


In [173]:
output_df_rs = output_df.set_index('DateTime').resample('15Min').mean().interpolate('linear')
#output_df_rs

In [175]:
print (par_text.ice_accretion_6hr)

None
